In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from huggingface_hub import login
login("Enter HF key here")


In [8]:
!pip install scikit-learn==1.1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 547.0 kB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "/mnt/Data/prabirmondal/anaconda3/envs/inv1/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/mnt/Data/prabirmondal/anaconda3/envs/inv1/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/mnt/Data/prabirmondal/anaconda3/envs/inv1/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 112, in ge

In [7]:
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from sklearn.metrics import accuracy_score
from tqdm import tqdm

input_dir = '/mnt/Data/prabirmondal/prabir/python_program/movie_sense/SRI_KG/Movie_sense_KG/Movie_sense_KG/yuvraj/yes no sample input'
output_dir = '/mnt/Data/prabirmondal/prabir/python_program/movie_sense/SRI_KG/Movie_sense_KG/Movie_sense_KG/yuvraj/yes no sample output'
report_path = os.path.join(output_dir, 'report.txt')

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", use_auth_token=True)

def calculate_accuracy(true_answers, predictions):
    correct = sum(1 for true, pred in zip(true_answers, predictions) if true == pred)
    return correct / len(true_answers) if true_answers else 0

def generate_response(question):
    prompt = (
        "Answer the question about the Hollywood movie with either 'yes' or 'no' only. "
        "Examples (DO NOT ANSWER THESE):\n"
        "Q: Is the earth round?\nA: yes\n"
        "Q: Can humans breathe underwater without equipment?\nA: no\n\n"
        "Now, answer this question:\n"
        f"Q: {question}"
    )
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=5, temperature=0.5, top_p=0.9)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip().lower()
    
    print(response)
    if "yes" in response:
        return "yes"
    elif "no" in response:
        return "no"
    else:
        return "uncertain"


all_accuracies = []
report_content = []

for filename in os.listdir(input_dir):
    if filename.endswith('.csv'):
        input_file_path = os.path.join(input_dir, filename)
        output_file_path = os.path.join(output_dir, filename)
        
        if os.path.exists(output_file_path):
            existing_data = pd.read_csv(output_file_path)
            file_accuracy = float(existing_data.iloc[-1, 2])  # Ensure accuracy is a float
            all_accuracies.append(file_accuracy)
            report_content.append(f"{filename}: {file_accuracy}")
            continue
        
        data = pd.read_csv(input_file_path)
        predictions = []
        true_answers = data['Answer'].tolist()
        
        for question in tqdm(data['Question'], desc=f"Processing {filename}"):
            response = generate_response(question)
            predictions.append(response)
        
        file_accuracy = accuracy_score(true_answers, predictions)
        all_accuracies.append(float(file_accuracy))  # Ensure accuracy is a float
        report_content.append(f"{filename}: {file_accuracy}")
        
        data['LLAMA_response'] = predictions
        data.loc[len(data)] = ["", "", file_accuracy]
        data.to_csv(output_file_path, index=False)

file_accuracy = calculate_accuracy(true_answers, predictions)
overall_accuracy = calculate_accuracy(overall_true_answers, overall_predictions)


with open(report_path, 'w') as f:
    f.write("\n".join(report_content))

print("Processing complete. Check the output directory for results and report.txt.")

RecursionError: maximum recursion depth exceeded